In [ ]:
!pip install ultralytics

In [4]:
import json
import os
from ultralytics import YOLO

In [2]:
model = YOLO("./weights/best.pt") # 학습이 끝난 후 가중치는 runs/detect/train/weights/best.pt 경로에 저장되어있음
result = model.predict(source="/home/ubuntu/storage2/dongbeen/Competition_task2/data/test_data/", imgsz=480, iou=0.5, conf=0.2, device=1, save_txt=True, save=True) # 테스트 이미지 경로 (절대 경로)


image 1/25 /home/ubuntu/storage2/dongbeen/Competition_task2/data/test_data/image_001.jpg: 480x480 6 ics, 5 resistors, 5.5ms
image 2/25 /home/ubuntu/storage2/dongbeen/Competition_task2/data/test_data/image_002.jpg: 320x480 1 ic, 113.8ms
image 3/25 /home/ubuntu/storage2/dongbeen/Competition_task2/data/test_data/image_003.jpg: 352x480 2 ics, 112.3ms
image 4/25 /home/ubuntu/storage2/dongbeen/Competition_task2/data/test_data/image_004.jpg: 320x480 3 ics, 1 capacitor, 11 resistors, 5.4ms
image 5/25 /home/ubuntu/storage2/dongbeen/Competition_task2/data/test_data/image_005.jpg: 320x480 1 ic, 4.7ms
image 6/25 /home/ubuntu/storage2/dongbeen/Competition_task2/data/test_data/image_006.jpg: 320x480 2 ics, 5.2ms
image 7/25 /home/ubuntu/storage2/dongbeen/Competition_task2/data/test_data/image_007.jpg: 480x480 4 ics, 8 capacitors, 14 resistors, 250.3ms
image 8/25 /home/ubuntu/storage2/dongbeen/Competition_task2/data/test_data/image_008.jpg: 480x480 3 ics, 4.7ms
image 9/25 /home/ubuntu/storage2/dongbe

In [5]:
# Yolo Format을 COCO Format으로 변환
def convert_yolo_to_coco_format(idx, line, img_width, img_height):
    parts = list(map(float, line.strip().split()))
    class_id = int(parts[0])
    x_center, y_center, width, height = parts[1:5]

    x = (x_center * img_width) - (width * img_width / 2)
    y = (y_center * img_height) - (height * img_height / 2)

    width *= img_width
    height *= img_height

    return {
        "id": idx,
        "image_id": None,
        "category_id": class_id,
        "bbox": [x, y, round(width, 2), round(height, 2)],
        "segmentation": [],
        "area": width * height,
        "iscrowd": 0,
        "attributes": {"occluded": False, "rotation": 0.0}
    }

In [10]:
with open("/home/ubuntu/storage2/dongbeen/Competition_task2/data/answer_sample.json", "r") as f: # 제공된 정답파일 양식 경로
    answer_data = json.load(f)
answer_images = {img['id']: img for img in answer_data["images"]}
txt_files_dir = './runs/detect/predict/labels/' # Yolo를 이용해 추론한 label의 경로
txt_files = [f for f in os.listdir(txt_files_dir) if f.endswith('.txt')]

In [11]:
# image_id와 파일 이름을 넣고 image_id를 기준으로 정렬
def get_image_id_from_filename(filename, images_data):
    base_name = os.path.splitext(filename)[0]
    for image in images_data:
        if image['file_name'].startswith(base_name):
            return image['id']
    return None

answer_data['annotations'] = []
new_annotations = []

for idx, txt_file in enumerate(txt_files):
    with open(os.path.join(txt_files_dir, txt_file), 'r') as f:
        lines = f.readlines()
    img_id = get_image_id_from_filename(txt_file, answer_data['images'])
    for line in lines:
        coco_ann = convert_yolo_to_coco_format(idx, line, answer_images[img_id]['width'], answer_images[img_id]['height'])
        coco_ann['image_id'] = img_id
        new_annotations.append(coco_ann)
        
answer_data['annotations'] += new_annotations
sorted_ann = sorted(new_annotations, key=lambda x: x['image_id'])

for i, ann in enumerate(sorted_ann):
    ann['id'] = i

In [12]:
# 제출 파일 생성
answer_data['annotations'] = sorted_ann
with open('answer.json', 'w') as f: # 제출 파일이 저장될 경로
    json.dump(answer_data, f)